In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import *

races_schema = StructType(fields = [
    StructField("raceID", IntegerType(),False),
    StructField("year", IntegerType(),True),
    StructField("round", IntegerType(),True),
    StructField("circuitid", IntegerType(),True),
    StructField("name", StringType(),True),
    StructField("date", DateType(),False),
    StructField("time", StringType(),True),
    StructField("url", StringType(), True)
])

races_df = spark.read \
.option("header",True) \
.schema(races_schema) \
.csv(f"{raw_folder_path}/races.csv")

In [0]:
display(races_df)

In [0]:
from pyspark.sql.functions import *
races_df.describe().show()

In [0]:
races_df.columns

In [0]:
races_df_transformed = races_df.select(
col('raceID').alias("race_id"),
col('year').alias("race_year"),
col('round'),
col('circuitid').alias("circuit_id"), 
col('name'),
col('date'),
col('time'))

In [0]:
races_df_transformed = add_ingestion_date(races_df_transformed) \
.withColumn("race_timestamp", to_timestamp(concat(col('date'),lit(' '), col('time')),'yyyy-MM-dd HH:mm:ss'))

In [0]:
display(races_df_transformed)

In [0]:
races_df_transformed = races_df_transformed.drop("date","time")

In [0]:
races_df_transformed.printSchema()

In [0]:
races_df_transformed.write.mode("overwrite").partitionBy('race_year').parquet(f"{processed_folder_path}/races")